In [1]:
# Force IPython to auto-flush outputs
%config Application.log_level = 'INFO' 
%config InteractiveShell.ast_node_interactivity = "all"


In [2]:
from langchain_openai import ChatOpenAI
import sys
import os
import io


In [3]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Single LLM Call
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,
    api_key=os.getenv("OPENAI_API_KEY"),  # best practice
    request_timeout=120,
    max_retries=5,
    verbose=False,
   
)

messages = [
    ("assistant", "You are a helpful assistant."),
    ("human", "Write me a haiku about autumn leaves.")
]



res = llm.invoke(messages)   # list-of-conversations -> ChatResult

print(res.content) 

print(res)

Crimson leaves descend,  
Whispers of the cooling breeze,  
Autumn’s soft farewell.
content='Crimson leaves descend,  \nWhispers of the cooling breeze,  \nAutumn’s soft farewell.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 26, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CIdVoFw1dUCaHtVymQCTKd1fMKZsD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e5f80354-8fbf-4a37-be43-1a2b04972a96-0' usage_metadata={'input_tokens': 26, 'output_tokens': 20, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'aime la programmation."

In [ ]:
if False:
    # Batched LLM Calls
    from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

    messages = [
        ("assistant", "You are a helpful assistant."),
        ([HumanMessage("Write me a haiku about autumn leaves.")])
    ] * 5

    completion = llm.batch(messages)  # list-of-list-of-conversations -> ChatResult

    print(completion)

    for i, choice in enumerate(completion):
        print(f"\n\nCHOICE {i+1}\n")
        print(choice.content)


[AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CIdVqEXt3LfwKUvDbF8727DFSb60W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e1d461b6-56f7-4f6c-b7b0-bfcc1a75a4e4-0', usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='Crimson leaves descend,  \nWhispering tales to the breeze,  \nAutumn’s soft farewell.', additional_kwargs={'refusal': None}, response_metadata={'t

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "French",
        "input": "I love programming.",
    }
)

AIMessage(content="J'aime programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 26, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3502f4eb73', 'id': 'chatcmpl-CIdxhK63PXDJwRxcPYrbUc32331Gc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--782bb7b5-a841-4908-a517-547c7a75f9eb-0', usage_metadata={'input_tokens': 26, 'output_tokens': 4, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B653BD24E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B653BD0B60>, root_client=<openai.OpenAI object at 0x000001B653B42D50>, root_async_client=<openai.AsyncOpenAI object at 0x000001B653BD2360>, model_name='gpt-4.1', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
if False:

    # how to have an interactive real time conversation with the assistant?

    # Initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

    def ask_once(user_input: str):
        # Only system + user, no history
        messages = [
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content=user_input)
        ]
        response = llm.invoke(messages)
        return response.content

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = ask_once(user_input)
        i += 1
        print(f"Assistant: {answer}\n")

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?

What did you notice about this Assistant?

In [ ]:
if False:

    # What is diffeernt about this version?

    # initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

    # start with a system prompt
    history = [SystemMessage(content="You are a helpful assistant.")]

    # function to ask user input and continue the conversation
    def chat_turn(user_input: str):
        # append user message
        history.append(HumanMessage(content=user_input))

        # call model with full history
        ai_msg = llm.invoke(history)  # returns an AIMessage
        #print(f"Assistant: {ai_msg.content}")

        # append assistant reply manually (so it's fed into the next turn)
        history.append(ai_msg)
        return ai_msg.content

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = chat_turn(user_input)
        i += 1
        print(f"Assistant: {answer}\n")

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?

In [ ]:
if False:
    # And about this version?

    # initialise model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, output_version="responses/v1")

    tool = {"type": "web_search_preview"}
    llm_with_tools = llm.bind_tools([tool])

    # start with a system prompt
    history = [SystemMessage(content="You are a helpful assistant.")]

    # function to ask user input and continue the conversation
    def chat_turn(user_input: str):
        # append user message
        history.append(HumanMessage(content=user_input))

        # call model with full history
        ai_msg = llm_with_tools.invoke(history)  # returns an AIMessage
        #print(f"Assistant: {ai_msg.content}")

        # append assistant reply manually (so it's fed into the next turn)
        history.append(ai_msg)
        return ai_msg.text()

    # Run a simple q&a session
    i=0
    while True:
        user_input = input(f"Q{i+1}: ")   # prompt user in Jupyter cell
        if user_input == "quit" or user_input == "exit" or user_input == "break" or not user_input.strip():        # allow empty string or quit or exit or break to break early
            break
        answer = chat_turn(user_input)
        i += 1
        print(f"Assistant: {answer}\n")
        print({answer})

        # Example questions are: where were the last olympics held?  who won the world cup in 2022?  Who wa the team captain? what is the capital of france? who is the president of the united states? what is the tallest mountain in the world?
        # who won the women's 100m  hurldes at the 2025 tokyo world athletics championship

In [10]:
print(history)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})]


In [11]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel


def get_weather(location: str) -> None:
    """Get weather at a location."""
    return "It's sunny."


class OutputSchema(BaseModel):
    """Schema for response."""

    answer: str
    justification: str


llm = ChatOpenAI(model="gpt-4.1")

structured_llm = llm.bind_tools(
    [get_weather],
    response_format=OutputSchema,
    strict=True,
)

# Response contains tool calls:
tool_call_response = structured_llm.invoke("What is the weather in SF?")

# structured_response.additional_kwargs["parsed"] contains parsed output
structured_response = structured_llm.invoke(
    "What weighs more, a pound of feathers or a pound of gold?"
)
print(structured_response.content)

{"answer":"A pound of feathers and a pound of gold both weigh the same: one pound.","justification":"The weight measurement 'pound' is the same unit for both, so a pound of any substance, including feathers and gold, is equal in weight. However, in commercial and scientific settings, gold is often measured using 'troy pounds' (which is about 373 grams) while feathers use 'avoirdupois pounds' (about 454 grams). If using these different systems, the avoirdupois pound (used for feathers) is heavier than the troy pound (used for gold). But when referring simply to 'a pound' as a common unit, they weigh the same."}


In [ ]:
# Example of a tool call
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather for a city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"],
            },
        },
    }
]

resp = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": "What's the weather in London?"}],
    tools=tools,
    tool_choice="auto",
)

print(resp.choices[0].message)
